In [ ]:
import setup_django

setup_django.init()

Current Working Directory: c:\Users\USER\Downloads\Compressed\recommender-main\src\nbs
Python Path: ['c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\src', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\.venv', '', 'c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\.venv\\Lib\\site-packages', 'c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\USER\\Downloads\\Compressed\\recommender-main\\.venv\\Lib\\site-packages\\Pythonwin']


In [6]:
import numpy as np
import pandas as pd
from django.contrib.auth import get_user_model
from django.db.models import F, Count, Sum, IntegerField
from movies.models import Movie
from ratings.models import Rating
import random

User = get_user_model()
print(Rating.objects.filter(user__username="cfe", value__gte=5).first())

None


In [ ]:
rating_obj = Rating.objects.filter(user__username="cfe", value__gte=5).first()
rating_obj.content_object

AttributeError: 'NoneType' object has no attribute 'content_object'

In [5]:
movie_id = rating_obj.content_object.id
value = rating_obj.value
user_id = rating_obj.user_id

AttributeError: 'NoneType' object has no attribute 'content_object'

In [ ]:
print(f"Movie {movie_id} was rated {value} by {user_id}")

In [ ]:
other_user_ids = (
    Rating.objects.filter(
        object_id=rating_obj.object_id,

        content_type=rating_obj.content_type,
        value__gte=rating_obj.value,
    )
    .exclude(user=rating_obj.user)
    .values_list("user_id", flat=True)
)



highly_rated = Rating.objects.filter(user_id__in=other_user_ids, value__gte=4)

In [ ]:
rec_users = []
rec_movies = []
for rating_obj in highly_rated:
    if rating_obj.user not in rec_users:
        rec_users.append(rating_obj.user)
    object_id = rating_obj.object_id
    if object_id not in rec_movies:
        rec_movies.append(object_id)

In [ ]:
print(len(rec_users), len(set(rec_movies)))

In [ ]:
import numpy as np

In [ ]:
# Write a list of 2 movies attributes
movie_1 = ["sci-fi", "comedy", "Stars latest action movie star"]
movie_2 = ["historical fiction", "drama", "Stars latest action movie star"]

# Write a list of a User's preferences
user_a = ["comedy", "sci-fi", "Stars latest action movie star"]
print(f"User has {len(user_a)} primary preferences")

# Total number of potential features
num_unique_features = len(set(movie_1 + movie_2))
print(f"These movies have {num_unique_features} unique features.")

Convert the user preferences list into a numerical score between -1 and 1

In [ ]:
my_preferences = np.array([0.98, 0.85, 0.75])

Convert movie attributes into a numerical score between -1 and 1 based on the following 3 attributes:

- Sci-Fi
- Comedy
- Stars latest action hero

In [ ]:
movie_attribute_scores = np.array([0.96, 0.99, 0.92])

movie_2_attribute_scores = np.array([-0.93, -0.45, 0.5])

Predict User's likelihood of enjoying either movie using Matrix multiplication.

In [ ]:
(my_preferences * movie_attribute_scores).sum()

In [ ]:
(my_preferences * movie_2_attribute_scores).sum()

In [ ]:
max_score = (np.array([1, 1, 1]) * np.array([1, 1, 1])).sum()
min_score = (np.array([1, 1, 1]) * np.array([-1, -1, -1])).sum()
print(max_score, min_score)

This notebook was inspired from Chapter 8 In [Deep Learning for Coders](https://amzn.to/3Bs48Sz) by Jeremy Howard & Sylvain Gugger

In [ ]:
ids = Movie.objects.all().popular()[:15]
most_active_users = (
    Rating.objects.all()
    .annotate(user_count=Count("user"))
    .order_by("-user_count")
    .values_list("user_id")[:5_000]
)

ratings_qs = (
    Rating.objects.filter(
        object_id__in=ids, user_id__in=most_active_users, active=True, value__gte=0
    )
    .annotate(movie=F("object_id"))
    .values("user", "value", "movie")
)

df = pd.DataFrame(ratings_qs)

In [ ]:
df.pivot_table(index="user", columns=["movie"], values="value", fill_value="")[:15]

In [ ]:
movie_1 = np.array([1, 1, 1, 1, 1])
user_1 = np.array([1, 1, 1, 1, 1])
(user_1 * movie_1).sum()

```python
movie_1_features = np.array(["x", "y", "z", "xx", "yy"])
user_1_preferences = np.array(["x", "y", "z", "xx", "yy"])
assert (user_1_preferences * movie_1_features).sum() == 5
```

These are called latent features. They are hidden and it's what the machine learning model will try and find for us.